In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from statannotations.Annotator import Annotator
from statannot import add_stat_annotation

import statannot
import json
import glob
import random
import nmi
import string

from pathlib import Path
from matplotlib.ticker import FormatStrFormatter
from helps import *

import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
plt.rcParams['font.size'] = 15
plt.rcParams["xtick.labelsize"]=13
plt.rcParams["ytick.labelsize"]=13
plt.rcParams["axes.titlesize"]=15
plt.rcParams["figure.dpi"]=600
plt.rcParams["savefig.format"]="pdf"
plt.rcParams["savefig.bbox"]="tight"

# Info-ENS

In [ ]:
info=pd.read_csv("HelperFiles/mart_export.txt",sep="\t", index_col=0)
info = info[~info.index.duplicated(keep='first')]
print(set(info["Gene type"]))
ok=info[(info["Gene type"] == "protein_coding") ^ (info["Gene type"] == "lincRNA")]
info.head()

In [ ]:
gene_names=ok["Gene name"].to_numpy()
ribs=[]
for name in gene_names:
    if name[:3]=="RPL" or name[:3]=="RPS":
        ribs.append(name)
len(ribs)

In [ ]:
temp=info
temp["ENS"]=temp.index
temp=temp.set_index("Gene name")
temp["Gene name"]=temp.index
temp["Gene type"].loc[ribs]="RIB-pt"
temp.index.names=["Name"]
temp.set_index("ENS", inplace=True)
temp.to_csv("HelperFiles/ENS-Info.txt",sep="\t")
temp

In [ ]:
info=pd.read_csv("HelperFiles/ENS-Info.txt",sep="\t", index_col=0)
info = info[~info.index.duplicated(keep='first')]
print(set(info["Gene type"]))
ok=info[(info["Gene type"] == "protein_coding") ^ (info["Gene type"] == "lincRNA")]
info.head()

In [ ]:
gene_names=ok["Gene name"].to_numpy()
mt=[]
for name in gene_names:
    if name[:3]=="MT-":
        mt.append(name)
len(mt)

In [ ]:
temp=info
temp["ENS"]=temp.index
temp=temp.set_index("Gene name")
temp["Gene name"]=temp.index
temp["Gene type"].loc[mt]="MT"
temp.index.names=["Name"]
temp.set_index("ENS", inplace=True)
temp.to_csv("HelperFiles/ENS-Info.txt",sep="\t")
temp

In [ ]:
info=pd.read_csv("HelperFiles/ENS-Info.txt",sep="\t", index_col=0)
info = info[~info.index.duplicated(keep='first')]
print(set(info["Gene type"]))
ok=info[info["Gene type"] == "lincRNA"]
info.head()

In [ ]:
gene_names=ok["Gene name"].to_numpy()
mir=[]
for name in gene_names:
    if name[:3]=="MIR" or name[:3]=="hsa":
        mir.append(name)
len(mir)

In [ ]:
mir

In [ ]:
to_change=["MIR296","MIR24-2","MIR940","MIR146A","MIR219-2","MIR1302-11","MIR4313",
           "MIR145","MIR194-2","MIR194-2",'MIR3179-1', 'MIR3180-1', 'MIR4453', 'MIR3180-4', 'MIR3179-2',
 'MIR3180-2','MIR3180-3', 'MIR3179-3','MIR2117', 'MIR29A', 'MIR451B', 'MIR1302-2','MIR1587',
 'MIR1302-10','MIR22HG', 'hsa-mir-1253', 'hsa-mir-7515','hsa-mir-8072','MIR371B', 'MIR378D2',]

In [ ]:
to_change=["MIR296","MIR24-2","MIR940","MIR146A","MIR219-2","MIR1302-11","MIR4313",
           "MIR145","MIR194-2","MIR194-2",'MIR3179-1', 'MIR3180-1', 'MIR4453', 'MIR3180-4', 'MIR3179-2',
 'MIR3180-2','MIR3180-3', 'MIR3179-3','MIR2117', 'MIR29A', 'MIR451B', 'MIR1302-2','MIR1587',
 'MIR1302-10','MIR22HG', 'hsa-mir-1253', 'hsa-mir-7515','hsa-mir-8072','MIR371B', 'MIR378D2',]

temp=info
temp["ENS"]=temp.index
temp=temp.set_index("Gene name")
temp["Gene name"]=temp.index
temp["Gene type"].loc[to_change]="miRNA"
temp.index.names=["Name"]
temp.set_index("ENS", inplace=True)
temp.to_csv("HelperFiles/ENS-Info.txt",sep="\t")
temp

# mRNA vs lncRNA CV

In [ ]:
df=pd.read_csv("Datasets/Grosselin2019/Grosselin-sc-RNA-raw.csv.gz", index_col=0)
print(df.shape)
df.head()

In [ ]:
info=pd.read_csv("HelperFiles/ENS-Info.txt", sep="\t",index_col=0)
info = info[~info.index.duplicated(keep='first')]
print(info.shape)
info.head()

In [ ]:
info=info.loc[(info[info["Gene type"]=="protein_coding"]).index ^ (info[info["Gene type"]=="lincRNA"]).index]
info=info.loc[intersection([info.index, df.index])]
df=df.loc[intersection([info.index, df.index])]
df=df.loc[info.index]
info.shape, df.shape

In [ ]:
df=df*1e4/df.sum()
df=np.log(1+df)

In [ ]:
pd.set_option("display.precision", 10)
info["Mean"]=df.T.mean()
info["STD"]=df.T.std()
info["STN"]=info["Mean"]/info["STD"]
info["VMR"]=info["STD"]**2/info["Mean"]
info["CV"]=info["STD"]/info["Mean"]

In [ ]:
info["log(STN)"]=np.log(info["STN"])
info["log(VMR)"]=np.log(info["VMR"])
info["log(CV)"]=np.log(info["CV"])

In [ ]:
info.replace(to_replace="protein_coding", value="mRNA", inplace=True)
info.replace(to_replace="lincRNA", value="lncRNA", inplace=True)

In [ ]:
stat="log(CV)"
pv=scipy.stats.mannwhitneyu(info[info["Gene type"]=="lncRNA"][stat].dropna(),
                   info[info["Gene type"]=="mRNA"][stat].dropna())[1]
pv

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(10,9), dpi=600)
order=("mRNA","lncRNA")
sns.violinplot(data=info, x="Gene type", y=stat, ax=axs, 
               palette=["green","violet"], order=order, saturation=0.65, cut=0)
test_results = add_stat_annotation(axs, data=info, x="Gene type", y=stat,
                                   box_pairs=[order], order=order,
                                   test='Mann-Whitney', text_format='simple',
                                   comparisons_correction=None,
                                   text_annot_custom=["Mann-Whitney U-Test p-value < 1$^{-10}$"],
                                   loc='inside', verbose=2)
axs.set_ylabel("CV (log)", weight="bold", size=20)
axs.set_xticks([0,1],["mRNAs","lncRNAs"], size=20)
axs.set_xlabel(None)
title=f"Suppl_Fig_1"
plt.savefig(f"Results/Figures/{title}.png", dpi=600)
#plt.savefig(f"Results/Figures/{title}.pdf", dpi=600)
plt.show()

# MDL vs n_init

## hSBM

In [ ]:
dataset="mRNA"
df=pd.read_csv(f"Results/hSBM-{dataset}/hSBM-{dataset}.csv.gz",index_col=0)
print(df.shape)
df=df.sample(frac=1, axis=1)
df.head(3)

In [ ]:
for init in range(1,11):
    Path(f"Results/MDL vs n_init/{dataset}/N_init_{init}").mkdir(parents=True, exist_ok=True)
    for k in range(0,10):
        model = trisbm()
        model.make_graph_multiple_df(mRNA, [lncRNA])
        model.fit(n_init=init)
        model.save_levels(f"Results/MDL vs n_init/{dataset}/N_init_{init}/{dataset}-{k}")
        del model

In [ ]:
means, stds = [], []
df=pd.DataFrame(columns=["mdl","n_init"])
i=0
for init in range(1,11):
    files=sorted(glob.glob(f"Results/MDL vs n_init/hSBM-{dataset}/N_init_{init}/hSBM-{dataset}-*-entropy.txt"))
    i=0
    mls=[]
    for file in files:
        d=pd.DataFrame(index=range(0,len(files)), columns=df.columns)
        d["n_init"]=init
        with open(file, "r") as f:
            mls.append(float(f.readline()))
        i+=1
    d["mdl"]=mls
    df=pd.concat([df, d], axis=0)
    means.append(df[df["n_init"]==init].mdl.mean())
    stds.append(df[df["n_init"]==init].mdl.std())
df=df.reset_index().drop("index", axis=1)
means=np.array(means)
stds=np.array(stds)
df["n_init"]=df["n_init"].astype(int)
df["mdl"]=df["mdl"].astype(float)

In [ ]:
fig, ax=plt.subplots(1,1, figsize=(12,9), dpi=600)
ax=sns.boxplot(x="n_init", y="mdl", data=df, ax=ax, saturation=0.65)
ax.set_xlabel("Number of initializations", size=20, weight='bold')
ax.set_ylabel("MDL", size=20, weight='bold')
ax.tick_params(axis='both', which='major', labelsize=25, rotation=0)
plt.show()

## triBSM

In [ ]:
mRNA=pd.read_csv("Results/hSBM-mRNA/hSBM-mRNA.csv.gz", index_col=0)
lncRNA=pd.read_csv("Results/hSBM-lncRNA/hSBM-lncRNA.csv.gz", index_col=0)
lncRNA=lncRNA[mRNA.columns]
mRNA.shape, lncRNA.shape

In [ ]:
for init in range(1,11):
    for k in range(0,10):
        model = nsbm()
        model.make_graph_multiple_df(mRNA, [lncRNA])
        model.fit(n_init=7, verbose=False)
        path_to_save=f"Results/MDL vs n_init/triSBM-mRNA-lncRNA/N_init_{init}/triSBM-mRNA-lncRNA-{k}"
        Path(path_to_save).mkdir(parents=True, exist_ok=True)
        nmi.save_levels_nSBM(model,path_to_save)
        del model

In [ ]:
means, stds = [], []
df=pd.DataFrame(columns=["mdl","n_init"])
i=0
for init in range(1,11):
    files=sorted(glob.glob(f"Results/MDL vs n_init/triSBM-mRNA-lncRNA/N_init_{init}/triSBM-mRNA-lncRNA-*-entropy.txt"))
    i=0
    mls=[]
    for file in files:
        d=pd.DataFrame(index=range(0,len(files)), columns=df.columns)
        d["n_init"]=init
        with open(file, "r") as f:
            mls.append(float(f.readline()))
        i+=1
    d["mdl"]=mls
    df=pd.concat([df, d], axis=0)
    means.append(df[df["n_init"]==init].mdl.mean())
    stds.append(df[df["n_init"]==init].mdl.std())
df=df.reset_index().drop("index", axis=1)
means=np.array(means)
stds=np.array(stds)
df["n_init"]=df["n_init"].astype(int)
df["mdl"]=df["mdl"].astype(float)

In [ ]:
fig, ax=plt.subplots(1,1, figsize=(12,9))
ax=sns.boxplot(x="n_init", y="mdl", data=df, ax=ax, saturation=0.65)
ax.set_xlabel("Number of initializations", size=20, weight='bold')
ax.set_ylabel("MDL", size=20, weight='bold')
ax.tick_params(axis='both', which='major', labelsize=25, rotation=0)
plt.show()

## Both

In [ ]:
dataset="mRNA"
df=pd.read_csv(f"Results/hSBM-{dataset}/hSBM-{dataset}.csv.gz",index_col=0)
print(df.shape)
df=df.sample(frac=1, axis=1)
df.head(3)

In [ ]:
means, stds = [], []
df=pd.DataFrame(columns=["mdl","n_init"])
i=0
for init in range(1,11):
    files=sorted(glob.glob(f"Results/MDL vs n_init/hSBM-{dataset}/N_init_{init}/hSBM-{dataset}-*-entropy.txt"))
    i=0
    mls=[]
    for file in files:
        d=pd.DataFrame(index=range(0,len(files)), columns=df.columns)
        d["n_init"]=init
        with open(file, "r") as f:
            mls.append(float(f.readline()))
        i+=1
    d["mdl"]=mls
    df=pd.concat([df, d], axis=0)
    means.append(df[df["n_init"]==init].mdl.mean())
    stds.append(df[df["n_init"]==init].mdl.std())
df=df.reset_index().drop("index", axis=1)
means=np.array(means)
stds=np.array(stds)
df["n_init"]=df["n_init"].astype(int)
df["mdl"]=df["mdl"].astype(float)

In [ ]:
data={}
data["hSBM mRNAs"]=df

In [ ]:
means, stds = [], []
df=pd.DataFrame(columns=["mdl","n_init"])
i=0
for init in range(1,11):
    files=sorted(glob.glob(f"Results/MDL vs n_init/triSBM-mRNA-lncRNA/N_init_{init}/triSBM-mRNA-lncRNA-*-entropy.txt"))
    i=0
    mls=[]
    for file in files:
        d=pd.DataFrame(index=range(0,len(files)), columns=df.columns)
        d["n_init"]=init
        with open(file, "r") as f:
            mls.append(float(f.readline()))
        i+=1
    d["mdl"]=mls
    df=pd.concat([df, d], axis=0)
    means.append(df[df["n_init"]==init].mdl.mean())
    stds.append(df[df["n_init"]==init].mdl.std())
df=df.reset_index().drop("index", axis=1)
means=np.array(means)
stds=np.array(stds)
df["n_init"]=df["n_init"].astype(int)
df["mdl"]=df["mdl"].astype(float)

In [ ]:
data["multibranch SBM mRNAs-lncRNAs"]=df

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(22,9), dpi=600)
for i,key in zip([0,1],list(data.keys())):
    axs[i]=sns.boxplot(x="n_init", y="mdl", data=data[key], ax=axs[i], saturation=0.65)
    axs[i].set_xlabel(f"{key}\nNumber of initializations", size=20, weight='bold')
    axs[i].set_ylabel("MDL", size=20, weight='bold')
    axs[i].tick_params(axis='both', which='major', labelsize=25, rotation=0)
    axs[i].text(-0.065, 1, string.ascii_uppercase[i],
                 transform=axs[i].transAxes, size=20, weight='bold',rotation=0)

fig.subplots_adjust(wspace=0.25)    
title=f"Suppl_Fig_2"
plt.savefig(f"Results/Figures/{title}.png", dpi=600)
#plt.savefig(f"Results/Figures/{title}.pdf", dpi=600)
plt.show()

# Enrichment test

## MSigDB

In [ ]:
files=sorted(glob.glob("Datasets/DatabasesEnrichmentTest/MSigDB/*"))
files

In [ ]:
MSigDB={}
for file in files:
    with open(file) as json_file:
        dic = json.load(json_file)
    for key in dic.keys():
        coll=file[38:-16]
        MSigDB[f"{coll}_{key}"]=dic[key]["geneSymbols"]

In [ ]:
with open('HelperFiles/MSigDB.json', 'w') as f:
    json.dump(MSigDB, f)
len(MSigDB.keys())

In [ ]:
U=flat_list([MSigDB[key] for key in MSigDB.keys()])
len(U), U[:3]

In [ ]:
intersections=[len(intersection([U, MSigDB[key]])) for key in MSigDB.keys()]
len(intersections), np.max(intersections), np.min(intersections)

In [ ]:
MSigDB_inter=pd.DataFrame(np.array(intersections)/len(U), columns=["Relative intersection with universe"])
MSigDB_inter["Gene set"]=MSigDB.keys()
MSigDB_inter["Genes"]=MSigDB.values()
MSigDB_inter["Count"]=np.array(intersections)
MSigDB_inter["Class"]=[gs.split("_")[0] for gs in MSigDB_inter["Gene set"]]
MSigDB_inter.index=list(MSigDB_inter["Gene set"])
MSigDB_inter.to_csv("Results/GeneSet Databases/Intersections_MSigDB.csv")

In [ ]:
with open("HelperFiles//MSigDB.json") as json_file:
    MSigDB = json.load(json_file)
len(MSigDB.keys())

In [ ]:
MSigDB_inter=pd.read_csv("Results/GeneSet Databases/Intersections_MSigDB.csv", index_col=0)
print(MSigDB_inter.shape)
MSigDB_inter.head()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(16, 8), sharey=True)
d=sns.histplot(MSigDB_inter, x="Relative intersection with universe", ax=axs[0],
               stat="probability", cumulative=True)
for rectangle in d.patches:
    if rectangle.get_x() >= 0.05 and rectangle.get_x() <= 0.25:
        rectangle.set_facecolor('teal')
    elif rectangle.get_x() >= 0.25:
        rectangle.set_facecolor('red')    
axs[0].set_xlabel("Intersection MSigDB gene sets with universe [number of genes]", weight="bold")

sns.histplot(MSigDB_inter, x="Count", ax=axs[1], stat="probability", cumulative=True)

plt.show()

In [ ]:
sns.boxplot(data=MSigDB_inter, x="Relative intersection with universe", y="Class")
plt.xscale("log")
plt.show()

In [ ]:
to_keep=[]
for clas in sorted(set(MSigDB_inter["Class"])):
    print(clas)
    df=MSigDB_inter[MSigDB_inter["Class"]==clas]
    lists=[df.iloc[i].Genes.split(";") for i in range(len(df))]
    lists=[MSigDB[gs] for gs in df["Gene set"]]
    tot=flat_list(lists)
    inters=[len(intersection([tot,l])) for l in lists]
    df["inter"]=np.array(inters)/len(tot)
    d=sns.histplot(data=df, x="inter", stat="probability", cumulative=True, bins="fd")
    for rectangle in d.patches:
        if rectangle.get_x() >= 0.05 and rectangle.get_x() <= 0.15:
            rectangle.set_facecolor('teal')
        elif rectangle.get_x() >= 0.15:
            rectangle.set_facecolor('red')    
    plt.xlabel(f"{clas} intersection with relative universe")
    plt.show()
    to_keep.append(df[df["inter"]<=0.15].index)
    print(clas, df["inter"].max(), df.shape, df[df["inter"]<=0.15].shape)

In [ ]:
len(flat_list(to_keep)) == len(MSigDB)

## lncSEA

In [ ]:
files=sorted(glob.glob("Datasets/DatabasesEnrichmentTest/lncSEA/*csv.gz"))
files

In [ ]:
dfs=[pd.read_csv(file, sep="\t", header=0,
                 names=["Class","Sub Class","Set","LncRNA","Count"], index_col=0) for file in files]

In [ ]:
df_tot=pd.concat(dfs, axis=0)
df_tot.reset_index(drop=True, inplace=True)
df_tot['Class'] = df_tot['Class'].str.replace('_','')
df_tot.index=["_".join([df_tot.loc[line]["Class"],df_tot.loc[line]["Sub Class"], df_tot.loc[line]["Set"]])
         for line in df_tot.index]
df_tot = df_tot[~df_tot.index.duplicated(keep='first')]
print(df_tot.shape)
df_tot.head()

In [ ]:
lncSEA = {key: value.split(";") for key, value in zip(df_tot.index, df_tot.LncRNA)}
print(len(lncSEA.keys()))

In [ ]:
with open('HelperFiles/lncSEA.json', 'w') as f:
    json.dump(lncSEA, f)

In [ ]:
U=flat_list([lncSEA[key] for key in lncSEA.keys()])
len(U), U[:3]

In [ ]:
df_tot["Relative intersection with universe"]=df_tot["Count"]/len(U)

In [ ]:
df_tot.to_csv("Results/GeneSet Databases/Intersections_lncSEA.csv")

In [ ]:
with open("HelperFiles/lncSEA.json") as json_file:
    lncSEA = json.load(json_file)
len(lncSEA.keys())

In [ ]:
lncSEA_inter=pd.read_csv("Results/GeneSet Databases/Intersections_lncSEA.csv", index_col=0)
lncSEA_inter.head()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(16, 8), sharey=True)
d=sns.histplot(lncSEA_inter, x="Relative intersection with universe", ax=axs[0],
               stat="probability", cumulative=True)
for rectangle in d.patches:
    if rectangle.get_x() >= 0.05 and rectangle.get_x() <= 0.25:
        rectangle.set_facecolor('teal')
    elif rectangle.get_x() >= 0.25:
        rectangle.set_facecolor('red')    
axs[0].set_xlabel("Intersection lncSEA gene sets with universe [number of genes]", weight="bold")
sns.histplot(lncSEA_inter, x="Count", ax=axs[1], stat="probability", cumulative=True)
plt.show()

In [ ]:
sns.boxplot(data=lncSEA_inter, x="Relative intersection with universe", y="Class")
plt.xscale("log")
plt.show()

In [ ]:
to_keep=[]
for clas in sorted(set(lncSEA_inter["Class"])):
    print(clas)
    df=lncSEA_inter[lncSEA_inter["Class"]==clas]
    lists=[df.iloc[i].Genes.split(";") for i in range(len(df))]
    lists=[lncSEA[gs] for gs in df["Gene set"]]
    tot=flat_list(lists)
    inters=[len(intersection([tot,l])) for l in lists]
    df["inter"]=np.array(inters)/len(tot)
    d=sns.histplot(data=df, x="inter", stat="probability", cumulative=True, bins="fd")
    for rectangle in d.patches:
        if rectangle.get_x() >= 0.05 and rectangle.get_x() <= 0.15:
            rectangle.set_facecolor('teal')
        elif rectangle.get_x() >= 0.15:
            rectangle.set_facecolor('red')    
    plt.xlabel(f"{clas} intersection with relative universe")
    plt.show()
    to_keep.append(df[df["inter"]<=0.15].index)
    print(clas, df.shape, df[df["inter"]<=0.15].shape)

In [ ]:
old_shapes=pd.DataFrame(columns=["shape", "list_type"])
old_shapes["list_type"]=sorted(list(set(lncSEA_inter["Class"])))
old_shapes["shape"]=[lncSEA_inter[lncSEA_inter["Class"]==clas].shape[0] for clas in old_shapes["list_type"]]

shapes=pd.DataFrame(columns=["shape", "list_type"])
shapes["list_type"]=sorted(list(set(lncSEA_inter["Class"])))
shapes["shape"]=[lncSEA_inter[lncSEA_inter["Class"]==clas].loc[to_keep[i]].shape[0] 
                 for clas, i in zip(shapes["list_type"], range(0,len(to_keep)))]

In [ ]:
shapes.sort_values(by="shape", inplace=True)
old_shapes.sort_values(by="shape", inplace=True)
sns.scatterplot(data=shapes, x="shape", y="list_type", color="blue", label="Length after filtering")
sns.scatterplot(data=old_shapes, x="shape", y="list_type", color="red", label="Length before filtering")
plt.xscale("log")
plt.show()

In [ ]:
red = {key: lncSEA[key] for key in flat_list(to_keep)}
len(red)

In [ ]:
with open('HelperFiles/lncSEA_red.json', 'w') as f:
    json.dump(red, f)

### Distance vs length

In [ ]:
lncSEA_inter=pd.read_csv("Results/GeneSet Databases/Intersections_lncSEA.csv", index_col=0)
lncSEA_inter.head(3)

In [ ]:
classes=["AccessibleChromatin","EQTL","Enhancer","Survival"]

In [ ]:
with open("HelperFiles/lncSEA.json") as json_file:
    lncSEA = json.load(json_file)
len(lncSEA.keys())

In [ ]:
for clas in classes:
    red = {key: lncSEA[key] for key in set(lncSEA_inter[lncSEA_inter["Class"]==clas].index)}
    inter=pd.DataFrame(columns=red.keys(), index=red.keys())
    union=pd.DataFrame(columns=red.keys(), index=red.keys())
    for row in inter.index:
        for col in inter.columns[list(inter.index).index(row):]:
            inter.at[row, col]=len(intersection([red[row],red[col]]))
            inter.at[col, row]=inter.at[row, col]
            union.at[row, col]=len(flat_list([red[row],red[col]]))
            union.at[col, row]=union.at[row, col]
    Path(f"Datasets/DatabasesEnrichmentTest/lncSEA/{clas}/").mkdir(parents=True, exist_ok=True)
    inter.to_csv(f"Datasets/DatabasesEnrichmentTest/lncSEA/{clas}/{clas}_inter.csv.gz", compression="gzip")
    union.to_csv(f"Datasets/DatabasesEnrichmentTest/lncSEA/{clas}/{clas}_union.csv.gz", compression="gzip")
    wiu=inter/union
    wiu.to_csv(f"Datasets/DatabasesEnrichmentTest/lncSEA/{clas}/{clas}_wiu.csv.gz", compression="gzip")

## Figures MSigDB and lncSEA

In [2]:
MSigDB_inter=pd.read_csv("Results/GeneSet Databases/Intersections_MSigDB.csv", index_col=0)
print(MSigDB_inter.shape)
MSigDB_inter.head()

(32880, 5)


,Relative intersection with universe,Gene set,Genes,Count,Class
B/c1_chr10p11,0.002969,B/c1_chr10p11,"['ABCD1P2', 'ACTR3BP5', 'AK3P5', 'ANKRD30A', '...",121,B/c1
B/c1_chr10p12,0.003312,B/c1_chr10p12,"['ABI1', 'ACBD5', 'ADIPOR1P1', 'AIFM1P1', 'AMD...",135,B/c1
B/c1_chr10p13,0.001546,B/c1_chr10p13,"['ACBD7', 'BEND7', 'BTBD7P1', 'C1QL3', 'CAMK1D...",63,B/c1
B/c1_chr10p14,0.001153,B/c1_chr10p14,"['ATP5F1C', 'CDC123', 'CELF2', 'CELF2-AS1', 'C...",47,B/c1
B/c1_chr10p15,0.002037,B/c1_chr10p15,"['ADARB2', 'ADARB2-AS1', 'AKR1C1', 'AKR1C2', '...",83,B/c1


In [3]:
with open("HelperFiles/MSigDB.json") as json_file:
    MSigDB = json.load(json_file)
len(MSigDB.keys())

32880

In [4]:
lncSEA_inter=pd.read_csv("Results/GeneSet Databases/Intersections_lncSEA.csv", index_col=0)
print(lncSEA_inter.shape)
lncSEA_inter.head()

(41365, 6)


,Class,Sub Class,Set,LncRNA,Count,Relative intersection with universe
AccessibleChromatin_PROXIMAL_ATAC_CellLine_BJ,AccessibleChromatin,PROXIMAL,ATAC_CellLine_BJ,RP11-245P10.6;CTB-50E14.4;RP11-588L15.2;CTD-23...,11224,0.191935
AccessibleChromatin_CLOSEST_ATAC_CellLine_BJ,AccessibleChromatin,CLOSEST,ATAC_CellLine_BJ,Y_RNA;RP11-588L15.2;CTD-2373H9.5;RP11-421L10.1...,11199,0.191508
AccessibleChromatin_Overlap_ATAC_CellLine_BJ,AccessibleChromatin,Overlap,ATAC_CellLine_BJ,PSMD6-AS2;AC097713.3;RP4-663N10.1;RP11-550I24....,5612,0.095968
AccessibleChromatin_CLOSEST_ATAC_CellLine_CAL-1,AccessibleChromatin,CLOSEST,ATAC_CellLine_CAL-1,KCNQ5-IT1;U47924.31;U3;snoU13;RP5-886K2.3;CTA-...,11675,0.199648
AccessibleChromatin_PROXIMAL_ATAC_CellLine_CAL-1,AccessibleChromatin,PROXIMAL,ATAC_CellLine_CAL-1,KCNQ5-IT1;U47924.31;U3;snoU13;RP5-886K2.3;CTD-...,11661,0.199408


In [5]:
with open("HelperFiles/lncSEA.json") as json_file:
    lncSEA = json.load(json_file)
len(lncSEA.keys())

41365

In [6]:
fig, axs = plt.subplots(3,2, figsize=(12,17), dpi=600)
axs=axs.flatten()

d=sns.histplot(MSigDB_inter, x="Relative intersection with universe",
               stat="probability", cumulative=True, ax=axs[0], color="green", alpha=0.65)
d=sns.histplot(lncSEA_inter, x="Relative intersection with universe",
               stat="probability", cumulative=True, ax=axs[1], color="violet", alpha=0.65)  

axs[0].set_xlabel("MSigDB gene sets relative intersection with universe", weight="bold")
axs[1].set_xlabel("lncSEA gene sets relative intersection with universe", weight="bold")
axs[0].set_ylabel("Probability", weight="bold")
title="Suppl_Fig_3"

classes=["AccessibleChromatin","EQTL","Enhancer","Survival"]
i=2
for clas in classes:
    
    wiu=pd.read_csv(f"Datasets/DatabasesEnrichmentTest/lncSEA/{clas}/{clas}_wiu.csv.gz", index_col=0)
    data=pd.DataFrame((1-wiu).mean(), columns=["Mean Jaccard distance"])
    data["Length gene set"]=[len(lncSEA[gs]) for gs in data.index]   

    sns.scatterplot(data=data, x="Length gene set", y="Mean Jaccard distance", ax=axs[i])
    
    axs[i].set_title(f"Class {clas}", fontweight="bold")
    axs[i].set_xlabel("Lentgh of gene sets", fontweight="bold")
    axs[i].set_ylabel("Mean Jaccard distance", fontweight="bold")

    i+=1
    
for i in range(0,6):
    axs[i].text(-0.065, 1, string.ascii_uppercase[i],
                 transform=axs[i].transAxes, size=20, weight='bold',rotation=0)

plt.tight_layout()

plt.savefig(f"Results/Figures/{title}.png", dpi=600)
#plt.savefig(f"Results/Figures/{title}.pdf", dpi=600)

plt.show()

In [ ]:
MSigDB_inter["Database"]="MSigDB"
lncSEA_inter["Database"]="lncSEA"
tot=pd.concat([MSigDB_inter, lncSEA_inter])
tot.head()

In [ ]:
tot["Log relative inters"]=np.log(tot["Relative intersection with universe"])

In [ ]:
sns.violinplot(data=tot,x="Log relative inters", y="Database")
plt.axvline(x=np.log(0.25))
plt.axvline(x=np.log(0.05))

# Multiple run for perfomances measurement

## hSBM

In [ ]:
dataset="mRNA-lncRNA"
df=pd.read_csv(f"Results/hSBM-{dataset}/hSBM-{dataset}.csv.gz",index_col=0)
print(df.shape)
df=df.sample(frac=1, axis=1)
df.head(3)

In [ ]:
for k in range(0,20):
    model = sbmtm()
    print(strftime("%Y-%m-%d %H:%M:%S", localtime()))
    model=sbmtm()
    model.make_graph_from_BoW_df(df)
    print(strftime("%Y-%m-%d %H:%M:%S", localtime()))
    model.fit(n_init=7)
    print(strftime("%Y-%m-%d %H:%M:%S", localtime()))
    path_to_save=f"Results/hSBM-{dataset}/hSBM/MultipleRun/hSBM-{dataset}-{k}/hSBM-{dataset}-{k}"
    Path(path_to_save).mkdir(parents=True, exist_ok=True)
    model.save_levels(path_to_save)
    del model

In [ ]:
labels=pd.DataFrame(index=df.columns)
labels["typehisto"]=[col[3:-19] for col in df.columns]
subtypes=list(sorted(set(labels.typehisto)))
labels.head()

In [ ]:
NMIs=[]
NMIsratio=[]
for level in [0,1,2]:
    for k in range(0,20):
        with open(f"Results/hSBM-{dataset}/hSBM/MultipleRun/hSBM-{dataset}-{k}/hSBM-{dataset}-{k}-cluster-level-{level}.txt") as f:
            clusters=json.load(f)
        cluster_df=pd.DataFrame.from_dict(clusters,orient="index")
        labels["hSBM"]="--"
        for i in range(len(clusters)):
            labels["hSBM"].loc[np.asarray(np.asarray(clusters[str(i)])[:,0])]=i
    
        NMI=np.around(nmi.compute_normalised_mutual_information(labels["typehisto"],labels[f"hSBM"]),decimals=3)
        NMIs.append(NMI)
        nmi_rand=0    
        for h in range(1000):
            a=labels[f"hSBM"].to_list()
            np.random.shuffle(a)
            nmi_rand+=nmi.compute_normalised_mutual_information(labels["typehisto"],a)/1000
        NMIsratio.append(np.round(NMI/nmi_rand,decimals=2))

In [ ]:
performances=pd.DataFrame(index=range(0,60))
performances["NMI"]=NMIs
performances["NMI/NMI*"]=NMIsratio
performances["level"]=["0" for i in range(20)] + ["1" for i in range(20)] + ["2" for i in range(20)]
performances.to_csv(f"Results/hSBM-{dataset}/hSBM-{dataset}_performances.csv")

## nSBM

In [ ]:
mRNA=pd.read_csv("Results/hSBM-mRNA/hSBM-mRNA.csv.gz", index_col=0)
lncRNA=pd.read_csv("Results/hSBM-lncRNA/hSBM-lncRNA.csv.gz", index_col=0)
lncRNA=lncRNA[mRNA.columns]
mRNA.shape, lncRNA.shape

In [ ]:
for k in range(0,20):
    model = nsbm()
    print(strftime("%Y-%m-%d %H:%M:%S", localtime()))
    model.make_graph_multiple_df(mRNA, [lncRNA])
    print(strftime("%Y-%m-%d %H:%M:%S", localtime()))
    model.fit(n_init=7, verbose=False)
    print(strftime("%Y-%m-%d %H:%M:%S", localtime()))
    path_to_save=f"Results/triSBM-mRNA-lncRNA/nSBM/MultipleRun/triSBM-mRNA-lncRNA-{k}/triSBM-mRNA-lncRNA-{k}"
    Path(path_to_save).mkdir(parents=True, exist_ok=True)
    nmi.save_levels_nSBM(model,path_to_save)
    del model

In [ ]:
labels=pd.DataFrame(index=mRNA.columns)
labels["typehisto"]=[col[3:-19] for col in mRNA.columns]
subtypes=list(sorted(set(labels.typehisto)))
labels.head()

In [ ]:
NMIs=[]
NMIsratio=[]
for level in [0,1,2]:
    for k in range(0,20):
        df_clu=pd.read_csv(f"Results/triSBM-mRNA-lncRNA/nSBM/MultipleRun/triSBM-mRNA-lncRNA-{k}/triSBM-mRNA-lncRNA-{k}-level-{level}-clusters.csv",
                       index_col=0) 
        labels=labels.loc[df_clu.columns]
        labels["hSBM"]=[str(np.array(df_clu[col]).argmax()) for col in df_clu.columns]
    
        NMI=np.around(nmi.compute_normalised_mutual_information(labels["typehisto"],labels[f"hSBM"]),decimals=3)
        NMIs.append(NMI)
        nmi_rand=0    
        for h in range(1000):
            a=labels[f"hSBM"].to_list()
            np.random.shuffle(a)
            nmi_rand+=nmi.compute_normalised_mutual_information(labels["typehisto"],a)/1000
        NMIsratio.append(np.round(NMI/nmi_rand,decimals=2))

In [ ]:
performances=pd.DataFrame(index=range(0,60))
performances["NMI"]=NMIs
performances["NMI/NMI*"]=NMIsratio
performances["level"]=["0" for i in range(20)] + ["1" for i in range(20)] + ["2" for i in range(20)]
performances.to_csv(f"Results/triSBM-mRNA-lncRNA/triSBM-mRNA-lncRNA_performances.csv")

## Perfomances comparison

In [ ]:
triSBMmRNAlncRNA=pd.read_csv("Results/triSBM-mRNA-lncRNA/triSBM-mRNA-lncRNA_performances.csv", index_col=0)
hSBMmRNAlncRNA=pd.read_csv("Results/hSBM-mRNA-lncRNA/hSBM-mRNA-lncRNA_performances.csv", index_col=0)

In [ ]:
triSBMmRNAlncRNA["method"]="triSBMmRNAlncRNA"
hSBMmRNAlncRNA["method"]="hSBMmRNAlncRNA"
diz={}
diz["triSBMmRNAlncRNA"]=triSBMmRNAlncRNA
diz["hSBMmRNAlncRNA"]=hSBMmRNAlncRNA
data={}
for key in list(diz.keys()):
    l = np.array(diz[key][diz[key].level==1]["NMI/NMI*"])
    l = l[(l>np.quantile(l,0.05)) & (l<np.quantile(l,0.95))].tolist()
    print(key, len(l))
    data[key]=l
temp=pd.DataFrame.from_dict(data)
temp.head()

In [ ]:
pv=scipy.stats.mannwhitneyu(temp["triSBMmRNAlncRNA"], temp["hSBMmRNAlncRNA"])
pv

In [ ]:
ax=sns.boxplot(data=temp.melt(), x="variable", y="value", saturation=0.65)
test_results = add_stat_annotation(ax, data=temp.melt(),
                                   x="variable", y="value",perform_stat_test=False,
                                   comparisons_correction=None,
                                   box_pairs=[("triSBMmRNAlncRNA", "hSBMmRNAlncRNA")],
                                   test=None, pvalues=[pv[1]],text_format='simple',
                                   text_annot_custom=[f"Mann-Whitney U-Test p-value: {str(np.round(pv[1],4))}"],
                                   loc='inside', verbose=2)
ax.set_xticks([0,1],["multibranch SBM mRNAs-lncRNAs","hSBM mRNAs-lncRNAs"])
ax.set_ylabel("NMI/NMI*", weight="bold")
ax.set_xlabel("\nMethod", weight="bold")
title=f"Fig_4"
plt.savefig(f"Results/Figures/{title}.png", dpi=600)
plt.show()

# Top expressed mRNAs and lncRNAs

In [ ]:
mRNA=pd.read_csv("Results/hSBM-mRNA/hSBM-mRNA.csv.gz", index_col=0)
lncRNA=pd.read_csv("Results/hSBM-lncRNA/hSBM-lncRNA.csv.gz", index_col=0)
lncRNA=lncRNA[mRNA.columns]
mRNA.shape, lncRNA.shape

In [ ]:
info=pd.read_csv("HelperFiles/ENS-Info.txt",index_col=0,sep="\t")

In [ ]:
mRNA=sc.AnnData(mRNA.T)
mRNA.var.index=ens_to_name_l(mRNA.var.index, info)
mRNA

In [ ]:
lncRNA=sc.AnnData(lncRNA.T)
lncRNA.var.index=ens_to_name_l(lncRNA.var.index, info)
lncRNA

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,10), dpi=600)
ax=sc.pl.highest_expr_genes(mRNA,ax=ax, show=False)
title=f"Top 30 expressed mRNA"
#plt.savefig(f"Results/Figures/{title}.png", dpi=600)
#plt.savefig(f"Results/Figures/{title}.pdf", dpi=600)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,10), dpi=600)
ax=sc.pl.highest_expr_genes(lncRNA,ax=ax, show=False)
title=f"Top 30 expressed lncRNA"
#plt.savefig(f"Results/Figures/{title}.png", dpi=600)
#plt.savefig(f"Results/Figures/{title}.pdf", dpi=600)
plt.show()

# hSBM vs nSBM with mRNA and lncRNA

In [ ]:
dfmlnc=pd.read_csv("Results/hSBM-mRNA-lncRNA/hSBM-mRNA-lncRNA.csv.gz", index_col=0)
labels=pd.read_csv("HelperFiles/All-datasets-labels.csv", index_col=0)
labels=labels.loc[dfmlnc.columns]
labels.shape

In [ ]:
level=1
with open(f"Results/hSBM-mRNA-lncRNA/hSBM/hSBM-mRNA-lncRNA-cluster-level-{level}.txt") as f:
    clusters=json.load(f)
cluster_df=pd.DataFrame.from_dict(clusters,orient="index")
labels["hSBM_m_lnc"]="--"
for i in range(len(clusters)):
    labels["hSBM_m_lnc"].loc[np.asarray(np.asarray(clusters[str(i)])[:,0])]=i
labels.hSBM_m_lnc=labels.hSBM_m_lnc.astype(int)

df_clu=pd.read_csv(f"Results/triSBM-mRNA-lncRNA/nSBM/triSBM-mRNA-lncRNA-level-{level}-clusters.csv",
                       index_col=0)
df_clu=df_clu[labels.index]
labels["nSBM"]=[int(df_clu[col].argmax()) for col in df_clu.columns]
subtypes=sorted(list(set(labels.typehisto)))
labels.head()

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(21,9), dpi=600, sharey=True)
ticks=["hSBM mRNAs-lncRNAs", "multibranch SBM mRNAs-lncRNAs"]

for col, ax in zip(["hSBM_m_lnc","nSBM"], [0,1]):
    labels["typehisto_1"]=pd.Series(list(labels["typehisto"])).astype('category').cat.codes.values    
    fraction_sites = pd.DataFrame(index=labels[col].unique(), columns=sorted(labels["typehisto_1"].unique())[::-1]).fillna(0)
    for sample in labels[[col,"typehisto_1"]].values:
        fraction_sites.at[sample[0],sample[1]] += 1

    fraction_sites = fraction_sites.sort_values(by=list(fraction_sites.columns), ascending=True)
    fraction_sites.columns=subtypes[::-1]
    fraction_sites.plot.bar(stacked=True, color=dict(zip(subtypes, nmi.set_colors(subtypes))),
                           width=1, alpha=0.65, ax=axs[ax])    
    
    axs[ax].set_xlabel(f"\nClustering {ticks[ax]}", size=25, weight='bold')
    axs[ax].set_ylabel("Number of cells", size=25, weight='bold')
    axs[ax].yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
    axs[ax].tick_params(axis='both', which='major', labelsize=25, rotation=0)
    
    legend_properties = {'weight':'bold', "size":"x-large"}
    if ax==0:
        axs[ax].legend(loc=(0.67,0.7), prop=legend_properties)
    else:
        axs[ax].get_legend().remove()        
    
    axs[ax].text(-0.055, 1, string.ascii_uppercase[ax],
                 transform=axs[ax].transAxes, size=20, weight='bold',rotation=0)
    axs[ax].set_xticklabels([i for i in range(len(fraction_sites))])
    ax+=1
title=f"Fig_3"
plt.savefig(f"Results/Figures/{title}.png", dpi=600)
plt.savefig(f"Results/Figures/{title}.pdf", dpi=600)
plt.show()